# Video Analytics Executive Dashboard

**Purpose:** Provide actionable insights for Communication Specialists, Data Scientists, and Senior Executives to understand video performance and content strategy effectiveness.

## Key Questions This Notebook Answers:
1. **Performance:** Which videos are performing best? What are the trends?
2. **Engagement:** Are viewers watching videos to completion? Where do they drop off?
3. **Content Strategy:** What video lengths and content types drive the most engagement?
4. **Platform Usage:** How are viewers accessing content (desktop vs mobile)?
5. **Content Lifecycle:** Which content is stale and needs attention?
6. **Account Comparison:** How do different channels/accounts compare?

In [ ]:
# Setup and Imports
import duckdb
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pathlib import Path

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:,.2f}'.format)
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

# Connect to DuckDB
DB_PATH = Path('../output/analytics.duckdb')
if not DB_PATH.exists():
    raise FileNotFoundError(f"Database not found at {DB_PATH}. Run the pipeline first.")

conn = duckdb.connect(str(DB_PATH), read_only=True)
print(f"Connected to: {DB_PATH}")
print(f"Database size: {DB_PATH.stat().st_size / (1024*1024):.1f} MB")

---
## 1. Executive Summary
High-level KPIs and trends at a glance.

In [ ]:
# Key Performance Indicators
kpi_query = """
SELECT
    COUNT(DISTINCT video_id) as total_videos,
    COUNT(DISTINCT channel) as total_channels,
    SUM(video_view) as total_views,
    ROUND(AVG(engagement_score), 2) as avg_engagement_score,
    ROUND(AVG(video_percent_viewed), 2) as avg_percent_viewed,
    MIN(date) as data_from,
    MAX(date) as data_to,
    COUNT(DISTINCT date) as days_of_data
FROM daily_analytics
"""
kpis = conn.execute(kpi_query).fetchdf()

print("="*60)
print("           EXECUTIVE SUMMARY - KEY METRICS")
print("="*60)
print(f"\n  Total Videos Tracked:     {kpis['total_videos'].iloc[0]:,}")
print(f"  Total Channels/Accounts:  {kpis['total_channels'].iloc[0]:,}")
print(f"  Total Video Views:        {kpis['total_views'].iloc[0]:,}")
print(f"  Avg Engagement Score:     {kpis['avg_engagement_score'].iloc[0]:.1f}%")
print(f"  Avg Percent Viewed:       {kpis['avg_percent_viewed'].iloc[0]:.1f}%")
print(f"\n  Data Period: {kpis['data_from'].iloc[0]} to {kpis['data_to'].iloc[0]}")
print(f"  ({kpis['days_of_data'].iloc[0]} days of data)")
print("="*60)

In [ ]:
# Monthly View Trends
monthly_query = """
SELECT
    DATE_TRUNC('month', date) as month,
    SUM(video_view) as total_views,
    COUNT(DISTINCT video_id) as unique_videos_viewed,
    ROUND(AVG(engagement_score), 2) as avg_engagement
FROM daily_analytics
WHERE video_view > 0
GROUP BY 1
ORDER BY 1
"""
monthly = conn.execute(monthly_query).fetchdf()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Views trend
axes[0].bar(monthly['month'].astype(str), monthly['total_views'], color='steelblue', alpha=0.8)
axes[0].set_title('Monthly Video Views', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Month')
axes[0].set_ylabel('Total Views')
axes[0].tick_params(axis='x', rotation=45)
axes[0].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))

# Engagement trend
axes[1].plot(monthly['month'].astype(str), monthly['avg_engagement'], marker='o', linewidth=2, color='coral')
axes[1].set_title('Monthly Average Engagement Score', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Month')
axes[1].set_ylabel('Engagement Score (%)')
axes[1].tick_params(axis='x', rotation=45)
axes[1].set_ylim(0, 100)

plt.tight_layout()
plt.show()

---
## 2. Top Performing Content
Which videos are driving the most views and engagement?

In [ ]:
# Top 20 Videos by Total Views
top_views_query = """
SELECT
    channel,
    video_id,
    MAX(name) as video_name,
    SUM(video_view) as total_views,
    ROUND(AVG(engagement_score), 1) as avg_engagement,
    ROUND(AVG(video_percent_viewed), 1) as avg_completion,
    MIN(date) as first_view_date,
    MAX(date) as last_view_date
FROM daily_analytics
WHERE video_view > 0
GROUP BY channel, video_id
ORDER BY total_views DESC
LIMIT 20
"""
top_videos = conn.execute(top_views_query).fetchdf()

print("\n" + "="*80)
print("                    TOP 20 VIDEOS BY TOTAL VIEWS")
print("="*80)
display(top_videos[['channel', 'video_name', 'total_views', 'avg_engagement', 'avg_completion']].head(20))

In [ ]:
# Top Videos by Engagement (minimum 100 views for statistical significance)
top_engagement_query = """
SELECT
    channel,
    video_id,
    MAX(name) as video_name,
    SUM(video_view) as total_views,
    ROUND(AVG(engagement_score), 1) as avg_engagement,
    ROUND(AVG(video_percent_viewed), 1) as avg_completion,
    ROUND(AVG(video_engagement_100), 1) as pct_watched_100
FROM daily_analytics
WHERE video_view > 0
GROUP BY channel, video_id
HAVING SUM(video_view) >= 100
ORDER BY avg_engagement DESC
LIMIT 20
"""
top_engagement = conn.execute(top_engagement_query).fetchdf()

print("\n" + "="*80)
print("         TOP 20 VIDEOS BY ENGAGEMENT (min 100 views)")
print("="*80)
display(top_engagement)

---
## 3. Engagement Analysis
Understanding viewer behavior: Where do viewers drop off? What drives completion?

In [ ]:
# Engagement Funnel Analysis
funnel_query = """
SELECT
    ROUND(AVG(video_engagement_1), 2) as started,
    ROUND(AVG(video_engagement_25), 2) as reached_25pct,
    ROUND(AVG(video_engagement_50), 2) as reached_50pct,
    ROUND(AVG(video_engagement_75), 2) as reached_75pct,
    ROUND(AVG(video_engagement_100), 2) as completed
FROM daily_analytics
WHERE video_view > 0
"""
funnel = conn.execute(funnel_query).fetchdf()

stages = ['Started (1%)', 'Reached 25%', 'Reached 50%', 'Reached 75%', 'Completed (100%)']
values = funnel.iloc[0].values

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.barh(stages[::-1], values[::-1], color=['#2ecc71', '#3498db', '#9b59b6', '#e74c3c', '#e67e22'][::-1])
ax.set_xlabel('Percentage of Viewers (%)', fontsize=12)
ax.set_title('Video Engagement Funnel\nWhere Do Viewers Drop Off?', fontsize=14, fontweight='bold')
ax.set_xlim(0, 100)

# Add value labels
for bar, val in zip(bars, values[::-1]):
    ax.text(val + 2, bar.get_y() + bar.get_height()/2, f'{val:.1f}%', 
            va='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

# Drop-off analysis
print("\n" + "="*60)
print("           ENGAGEMENT DROP-OFF ANALYSIS")
print("="*60)
print(f"\n  Start to 25%:   {values[0] - values[1]:.1f}% drop-off")
print(f"  25% to 50%:     {values[1] - values[2]:.1f}% drop-off")
print(f"  50% to 75%:     {values[2] - values[3]:.1f}% drop-off")
print(f"  75% to 100%:    {values[3] - values[4]:.1f}% drop-off")
print(f"\n  Overall Completion Rate: {values[4]:.1f}%")

---
## 4. Content Strategy Analysis
What content characteristics drive better performance?

In [ ]:
# Video Length vs Engagement
length_query = """
SELECT
    CASE
        WHEN video_duration <= 60 THEN '1. Under 1 min'
        WHEN video_duration <= 180 THEN '2. 1-3 min'
        WHEN video_duration <= 300 THEN '3. 3-5 min'
        WHEN video_duration <= 600 THEN '4. 5-10 min'
        WHEN video_duration <= 1200 THEN '5. 10-20 min'
        WHEN video_duration <= 1800 THEN '6. 20-30 min'
        ELSE '7. Over 30 min'
    END as duration_bucket,
    COUNT(DISTINCT video_id) as num_videos,
    SUM(video_view) as total_views,
    ROUND(AVG(engagement_score), 1) as avg_engagement,
    ROUND(AVG(video_engagement_100), 1) as completion_rate
FROM daily_analytics
WHERE video_view > 0 AND video_duration > 0
GROUP BY 1
ORDER BY 1
"""
length_analysis = conn.execute(length_query).fetchdf()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Views by duration
axes[0].bar(length_analysis['duration_bucket'], length_analysis['total_views'], color='steelblue', alpha=0.8)
axes[0].set_title('Total Views by Video Duration', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Video Duration')
axes[0].set_ylabel('Total Views')
axes[0].tick_params(axis='x', rotation=45)
axes[0].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))

# Completion rate by duration
axes[1].bar(length_analysis['duration_bucket'], length_analysis['completion_rate'], color='coral', alpha=0.8)
axes[1].set_title('Completion Rate by Video Duration', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Video Duration')
axes[1].set_ylabel('Completion Rate (%)')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("              VIDEO LENGTH PERFORMANCE BREAKDOWN")
print("="*80)
display(length_analysis)

In [ ]:
# Content Type Performance (if available)
content_type_query = """
SELECT
    COALESCE(video_content_type, 'Unclassified') as content_type,
    COUNT(DISTINCT video_id) as num_videos,
    SUM(video_view) as total_views,
    ROUND(AVG(engagement_score), 1) as avg_engagement,
    ROUND(AVG(video_engagement_100), 1) as completion_rate
FROM daily_analytics
WHERE video_view > 0
GROUP BY 1
HAVING SUM(video_view) >= 50
ORDER BY total_views DESC
LIMIT 15
"""
content_types = conn.execute(content_type_query).fetchdf()

if len(content_types) > 1:
    print("\n" + "="*80)
    print("              CONTENT TYPE PERFORMANCE")
    print("="*80)
    display(content_types)
else:
    print("\nNote: Content type data not available or not classified.")

---
## 5. Channel/Account Performance Comparison
How do different accounts perform against each other?

In [ ]:
# Channel Performance Overview
channel_query = """
SELECT
    channel,
    COUNT(DISTINCT video_id) as num_videos,
    SUM(video_view) as total_views,
    ROUND(SUM(video_view) * 1.0 / COUNT(DISTINCT video_id), 0) as views_per_video,
    ROUND(AVG(engagement_score), 1) as avg_engagement,
    ROUND(AVG(video_engagement_100), 1) as completion_rate
FROM daily_analytics
WHERE video_view > 0
GROUP BY channel
ORDER BY total_views DESC
"""
channels = conn.execute(channel_query).fetchdf()

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Views by channel
colors = plt.cm.husl(np.linspace(0, 1, len(channels)))
axes[0].barh(channels['channel'], channels['total_views'], color=colors)
axes[0].set_title('Total Views by Channel', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Total Views')
axes[0].xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))
axes[0].invert_yaxis()

# Engagement by channel
axes[1].barh(channels['channel'], channels['avg_engagement'], color=colors)
axes[1].set_title('Average Engagement by Channel', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Engagement Score (%)')
axes[1].set_xlim(0, 100)
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("              CHANNEL PERFORMANCE SUMMARY")
print("="*80)
display(channels)

---
## 6. Device & Platform Analysis
How are viewers accessing content?

In [ ]:
# Device Breakdown
device_query = """
SELECT
    SUM(views_desktop) as desktop,
    SUM(views_mobile) as mobile,
    SUM(views_tablet) as tablet,
    SUM(views_other) as other
FROM daily_analytics
"""
devices = conn.execute(device_query).fetchdf()

device_data = {
    'Desktop': devices['desktop'].iloc[0],
    'Mobile': devices['mobile'].iloc[0],
    'Tablet': devices['tablet'].iloc[0],
    'Other': devices['other'].iloc[0]
}

# Filter out zero values
device_data = {k: v for k, v in device_data.items() if v > 0}

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Pie chart
colors = ['#3498db', '#e74c3c', '#2ecc71', '#9b59b6']
axes[0].pie(device_data.values(), labels=device_data.keys(), autopct='%1.1f%%', 
            colors=colors[:len(device_data)], startangle=90)
axes[0].set_title('Views by Device Type', fontsize=14, fontweight='bold')

# Bar chart
axes[1].bar(device_data.keys(), device_data.values(), color=colors[:len(device_data)])
axes[1].set_title('Total Views by Device', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Views')
axes[1].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))

plt.tight_layout()
plt.show()

total_device_views = sum(device_data.values())
print("\n" + "="*60)
print("           DEVICE BREAKDOWN SUMMARY")
print("="*60)
for device, views in device_data.items():
    pct = views / total_device_views * 100
    print(f"  {device:12} {views:>12,} views ({pct:5.1f}%)")

In [ ]:
# Device Trends Over Time
device_trend_query = """
SELECT
    DATE_TRUNC('month', date) as month,
    SUM(views_desktop) as desktop,
    SUM(views_mobile) as mobile,
    SUM(views_tablet) as tablet
FROM daily_analytics
GROUP BY 1
ORDER BY 1
"""
device_trends = conn.execute(device_trend_query).fetchdf()

# Calculate percentages
device_trends['total'] = device_trends['desktop'] + device_trends['mobile'] + device_trends['tablet']
device_trends['desktop_pct'] = device_trends['desktop'] / device_trends['total'] * 100
device_trends['mobile_pct'] = device_trends['mobile'] / device_trends['total'] * 100

fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(device_trends['month'].astype(str), device_trends['desktop_pct'], marker='o', label='Desktop', linewidth=2)
ax.plot(device_trends['month'].astype(str), device_trends['mobile_pct'], marker='s', label='Mobile', linewidth=2)
ax.set_title('Device Usage Trend Over Time', fontsize=14, fontweight='bold')
ax.set_xlabel('Month')
ax.set_ylabel('Percentage of Views (%)')
ax.legend()
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.show()

---
## 7. Content Lifecycle Analysis
Identify stale content and optimization opportunities.

In [ ]:
# Stale Content - Videos not viewed recently but still in library
stale_query = """
SELECT
    channel,
    video_id,
    MAX(name) as video_name,
    MAX(dt_last_viewed) as last_viewed,
    SUM(video_view) as total_views,
    MAX(created_at)::DATE as created_date
FROM daily_analytics
WHERE dt_last_viewed IS NOT NULL
GROUP BY channel, video_id
HAVING MAX(dt_last_viewed)::DATE < CURRENT_DATE - INTERVAL '180 days'
ORDER BY total_views DESC
LIMIT 20
"""
try:
    stale_content = conn.execute(stale_query).fetchdf()
    if len(stale_content) > 0:
        print("\n" + "="*80)
        print("    STALE CONTENT ALERT: Videos Not Viewed in 180+ Days")
        print("    (Sorted by historical views - these had audience interest)")
        print("="*80)
        display(stale_content)
    else:
        print("\nNo stale content found (all videos viewed within 180 days).")
except:
    print("\nNote: dt_last_viewed data not available for stale content analysis.")

In [ ]:
# Recent Content Performance (Last 30 Days)
recent_query = """
SELECT
    channel,
    video_id,
    MAX(name) as video_name,
    SUM(video_view) as views_last_30d,
    ROUND(AVG(engagement_score), 1) as avg_engagement,
    MAX(created_at)::DATE as created_date
FROM daily_analytics
WHERE date >= CURRENT_DATE - INTERVAL '30 days'
  AND video_view > 0
GROUP BY channel, video_id
ORDER BY views_last_30d DESC
LIMIT 15
"""
recent_content = conn.execute(recent_query).fetchdf()

print("\n" + "="*80)
print("         TOP PERFORMING CONTENT (Last 30 Days)")
print("="*80)
display(recent_content)

---
## 8. Actionable Insights & Recommendations

In [ ]:
# Generate Automated Insights
print("\n" + "="*80)
print("         ACTIONABLE INSIGHTS & RECOMMENDATIONS")
print("="*80)

# Insight 1: Best performing video length
if len(length_analysis) > 0:
    best_length = length_analysis.loc[length_analysis['completion_rate'].idxmax()]
    print(f"\n1. OPTIMAL VIDEO LENGTH")
    print(f"   Highest completion rate: {best_length['duration_bucket']} ({best_length['completion_rate']:.1f}% completion)")
    print(f"   Recommendation: Focus on creating videos in this duration range for maximum engagement.")

# Insight 2: Channel opportunities
if len(channels) > 1:
    high_engagement_low_views = channels[channels['avg_engagement'] > channels['avg_engagement'].median()]
    high_engagement_low_views = high_engagement_low_views[high_engagement_low_views['total_views'] < channels['total_views'].median()]
    if len(high_engagement_low_views) > 0:
        print(f"\n2. UNDERUTILIZED HIGH-ENGAGEMENT CHANNELS")
        for _, row in high_engagement_low_views.iterrows():
            print(f"   - {row['channel']}: High engagement ({row['avg_engagement']:.1f}%) but lower views")
        print(f"   Recommendation: Increase promotion of content from these channels.")

# Insight 3: Device strategy
if device_data:
    mobile_pct = device_data.get('Mobile', 0) / total_device_views * 100 if total_device_views > 0 else 0
    print(f"\n3. MOBILE OPTIMIZATION")
    if mobile_pct > 30:
        print(f"   Mobile views: {mobile_pct:.1f}% of total traffic")
        print(f"   Recommendation: Ensure all videos are mobile-optimized. Consider vertical video formats.")
    else:
        print(f"   Mobile views: {mobile_pct:.1f}% of total traffic")
        print(f"   Recommendation: Desktop-first approach is appropriate. Consider mobile push to increase reach.")

# Insight 4: Engagement drop-off
if funnel is not None and len(funnel) > 0:
    first_drop = values[0] - values[1]
    print(f"\n4. ENGAGEMENT OPTIMIZATION")
    print(f"   Biggest drop-off: First 25% of video ({first_drop:.1f}% viewers leave)")
    print(f"   Recommendation: Focus on stronger video openings. First 30 seconds are critical.")

# Insight 5: Content freshness
print(f"\n5. CONTENT FRESHNESS")
if 'stale_content' in dir() and len(stale_content) > 0:
    print(f"   {len(stale_content)} videos haven't been viewed in 180+ days")
    print(f"   Recommendation: Review stale content for archival, update, or re-promotion.")
else:
    print(f"   Content library appears fresh - good content lifecycle management!")

print("\n" + "="*80)

---
## 9. Custom Query Playground
Use this section to run your own queries against the database.

In [ ]:
# Custom Query - Modify as needed
custom_query = """
-- Example: Find videos with high impressions but low play rate
SELECT
    channel,
    video_id,
    MAX(name) as video_name,
    SUM(video_impression) as impressions,
    SUM(video_view) as views,
    ROUND(SUM(video_view) * 100.0 / NULLIF(SUM(video_impression), 0), 2) as play_rate
FROM daily_analytics
WHERE video_impression > 0
GROUP BY channel, video_id
HAVING SUM(video_impression) >= 100
ORDER BY play_rate ASC
LIMIT 20
"""

custom_result = conn.execute(custom_query).fetchdf()
print("Low Play Rate Videos (High Impressions, Low Clicks):")
print("These videos may need better thumbnails or titles.\n")
display(custom_result)

In [ ]:
# Database Schema Reference
schema_query = """
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'daily_analytics'
ORDER BY ordinal_position
"""
schema = conn.execute(schema_query).fetchdf()
print("\nAvailable Columns in daily_analytics table:")
print("-" * 50)
for _, row in schema.iterrows():
    print(f"  {row['column_name']:40} {row['data_type']}")

### Fuzzy Video Search
Search for videos by name using fuzzy matching (finds similar names even with typos).

In [ ]:
# Fuzzy Video Search Function
def fuzzy_search_videos(search_term, top_n=10, min_score=0.6):
    """
    Search for videos using fuzzy string matching.
    
    Args:
        search_term: String to search for
        top_n: Number of results to return (default 10)
        min_score: Minimum similarity score 0-1 (default 0.6)
    
    Returns:
        DataFrame with top matching videos
    """
    query = f"""
    SELECT DISTINCT
        video_id,
        name,
        channel,
        jaro_winkler_similarity(name, '{search_term}') as similarity_score,
        SUM(video_view) OVER (PARTITION BY video_id) as total_views,
        ROUND(AVG(engagement_score) OVER (PARTITION BY video_id), 1) as avg_engagement,
        MAX(created_at) OVER (PARTITION BY video_id)::DATE as created_date
    FROM daily_analytics
    WHERE jaro_winkler_similarity(name, '{search_term}') >= {min_score}
    ORDER BY similarity_score DESC, total_views DESC
    LIMIT {top_n}
    """
    
    results = conn.execute(query).fetchdf()
    
    if len(results) == 0:
        print(f"No videos found matching '{search_term}' with score >= {min_score}")
        print("Try lowering min_score (e.g., 0.4) or using different search terms.")
        return None
    
    return results


# Example usage - modify search_term to search for different videos
search_term = "Quarterly Report"  # Change this to search for different videos
min_similarity = 0.5  # Adjust this (0.0 - 1.0) - lower = more results but less accurate

print(f"\n{'='*80}")
print(f"  FUZZY SEARCH RESULTS FOR: '{search_term}'")
print(f"  (Similarity threshold: {min_similarity})")
print(f"{'='*80}\n")

results = fuzzy_search_videos(search_term, top_n=10, min_score=min_similarity)

if results is not None:
    # Format similarity score as percentage for readability
    results['match_score'] = (results['similarity_score'] * 100).round(1).astype(str) + '%'
    display_cols = ['match_score', 'name', 'channel', 'total_views', 'avg_engagement', 'created_date']
    display(results[display_cols])
    
    print(f"\nFound {len(results)} matching video(s)")
    print(f"\nTip: Adjust 'search_term' and 'min_similarity' variables above to refine search.")

In [ ]:
# Close connection
conn.close()
print("\nDatabase connection closed.")
print("\n" + "="*80)
print("                    END OF EXECUTIVE REPORT")
print("="*80)